In [1]:
try:
    from tfx import v1 as tfx
    
    if tfx.__version__ != "1.4.0":
        raise ModuleNotFoundError
except ModuleNotFoundError:
    !pip install tfx==1.4.0

In [2]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip
%load_ext tensorboard

print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow version: {}'.format(tf.__version__))

TFX version: 1.4.0
TensorFlow version: 2.6.2


In [3]:
import os
from absl import logging
import uuid

logging.set_verbosity(logging.INFO)

PIPELINE_NAME = "iris"
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = os.path.join("data")
os.makedirs(DATA_ROOT, exist_ok=True)

if False:
    TENSORBOARD_DIR = os.path.join('/tmp', 'tensorboard', str(uuid.uuid1()))
    os.makedirs(TENSORBOARD_DIR, exist_ok=True)
else:
    TENSORBOARD_DIR = None

# Retrieve data

In [4]:
import urllib.request

_data_url = 'https://datahub.io/machine-learning/iris/r/iris.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")

urllib.request.urlretrieve(_data_url, _data_filepath)

!head {_data_filepath}

# Create TFX pipeline and run it

In [5]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
    # Brings data into the pipeline or otherwise joins/converts training data.
    example_gen = tfx.components.CsvExampleGen(
        input_base=data_root)

    # Computes statistics over data for visualization and example validation.
    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs['examples'])

    # Generate a schema based on your data statistics.
    # A schema defines the expected bounds, types, and properties of the features in your dataset.
    schema_gen = tfx.components.SchemaGen(
        statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

    # Performs anomaly detection based on statistics and data schema.
    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema'])

    # Transforms input data using preprocessing_fn in the 'module_file'.
    transform = tfx.components.Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        materialize=False,
        module_file=module_file)
    
    # Tunes the hyperparams using the tuner_fn method in the 'module_file'.
    tuner = tfx.components.Tuner(
        module_file=module_file,
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=tfx.proto.TrainArgs(num_steps=20),
        eval_args=tfx.proto.EvalArgs(num_steps=5))

    # Uses user-provided Python function that trains a model.
    trainer = tfx.components.Trainer(
        module_file=module_file,
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        transform_graph=transform.outputs['transform_graph'],
        hyperparameters=tuner.outputs['best_hyperparameters'],
        train_args=tfx.proto.TrainArgs(num_steps=2000),
        eval_args=tfx.proto.EvalArgs(num_steps=5),
        custom_config={'tensorboard_dir': TENSORBOARD_DIR}
    )
    
    # Get the latest blessed model for model validation.
    model_resolver = tfx.dsl.Resolver(
        strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
        model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
        model_blessing=tfx.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing)
    ).with_id('latest_blessed_model_resolver')

    # Uses TFMA to compute evaluation statistics over features of a model and
    # perform quality validation of a candidate model (compared to a baseline).
    eval_config = tfma.EvalConfig(
      model_specs=[
          tfma.ModelSpec(
              #signature_name='serving_default',
              label_key='class',
              preprocessing_function_names=['transform_features'])
      ],
      slicing_specs=[tfma.SlicingSpec()],
      metrics_specs=[
          tfma.MetricsSpec(metrics=[
              tfma.MetricConfig(
                  class_name='SparseCategoricalAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.6}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
          ])
      ])
    evaluator = tfx.components.Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)

    # Pushes the model to a filesystem destination.
    pusher = tfx.components.Pusher(
        model=trainer.outputs['model'],
        model_blessing=evaluator.outputs['blessing'],
        push_destination=tfx.proto.PushDestination(filesystem=tfx.proto.PushDestination
                                                   .Filesystem(base_directory=serving_model_dir))
    )

    components = [
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        tuner,
        trainer,
        model_resolver,
        evaluator,
        pusher,
    ]

    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path),
        components=components)

In [6]:
_module_file = 'iris_utils.py'

pipeline = _create_pipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    data_root=DATA_ROOT,
    module_file=_module_file,
    serving_model_dir=SERVING_MODEL_DIR,
    metadata_path=METADATA_PATH
)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.


In [7]:
if TENSORBOARD_DIR:
    %tensorboard --logdir {TENSORBOARD_DIR}

In [8]:
tfx.orchestration.LocalDagRunner().run(pipeline)

Trial 6 Complete [00h 00m 01s]
val_accuracy: 0.8999999761581421

Best val_accuracy So Far: 0.9100000262260437
Total elapsed time: 00h 00m 07s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Int', 'config': {'name': 'dnn_hidden_layer_0', 'default': None, 'conditions': [], 'min_value': 100, 'max_value': 150, 'step': 1, 'sampling': None}}, {'class_name': 'Int', 'config': {'name': 'dnn_hidden_layer_1', 'default': None, 'conditions': [], 'min_value': 50, 'max_value': 70, 'step': 1, 'sampling': None}}], 'values': {'dnn_hidden_layer_0': 145, 'dnn_hidden_layer_1': 69}}
INFO:absl:Best Hyperparameters are written to pipelines/iris/Tuner/best_hyperparameters/389/best_hyperparameters.txt.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 389 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'best_hyperparameters': [Artifact(artifact: uri: "pipelines/iris/Tuner/best_hyperparameters/389"
custom_properties {
  key: "name"
  value {
    string_value: "iris

Results summary
Results in pipelines/iris/Tuner/tensorboards/389/iris_tune_model
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 145
dnn_hidden_layer_1: 69
Score: 0.9100000262260437
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 116
dnn_hidden_layer_1: 66
Score: 0.8999999761581421
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 147
dnn_hidden_layer_1: 52
Score: 0.8899999856948853
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 108
dnn_hidden_layer_1: 68
Score: 0.8600000143051147
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 130
dnn_hidden_layer_1: 58
Score: 0.8500000238418579
Trial summary
Hyperparameters:
dnn_hidden_layer_0: 101
dnn_hidden_layer_1: 64
Score: 0.8500000238418579


INFO:absl:Going to run a new execution: ExecutionInfo(execution_id=390, input_dict={'hyperparameters': [Artifact(artifact: id: 674
type_id: 26
uri: "pipelines/iris/Tuner/best_hyperparameters/389"
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-29T16:30:37.469749:Tuner:best_hyperparameters:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.4.0"
  }
}
state: LIVE
create_time_since_epoch: 1638199864247
last_update_time_since_epoch: 1638199864247
, artifact_type: id: 26
name: "HyperParameters"
)], 'examples': [Artifact(artifact: id: 663
type_id: 15
uri: "pipelines/iris/CsvExampleGen/examples/383"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "file_format"
  value {
    string_value: "tfrecords_gzip"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:4753,xor_checksum:1638199836,sum_checks

INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:udf_utils.get_fn {'eval_args': '{\n  "num_steps": 5\n}', 'custom_config': '{"tensorboard_dir": null}', 'module_path': 'iris_utils@pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+464589b475aa987fc57e5ad103d16f4739603683b4ec37897b571a148401d7b2-py3-none-any.whl', 'train_args': '{\n  "num_steps": 2000\n}'} 'run_fn'
INFO:absl:Installing 'pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+464589b475aa987fc57e5ad103d16f4739603683b4ec37897b571a148401d7b2-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/bin/python3.8', '-m', 'pip', 'install', '--target', '/tmp/tmp9o87o2j2', 'pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+464589b475aa987fc57e5ad103d16f4739603683b4ec37897b571a148401d7b2-py3-none-any.whl']


...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f2ef81bec40> and <keras.engine.input_layer.InputLayer object at 0x7f2ef81b2a00>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f2ef81bec40> and <keras.engine.input_layer.InputLayer object at 0x7f2ef81b2a00>).
INFO:absl:Evaluation complete. Results written to pipelines/iris/Evaluator/evaluation/391.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result False written to pipelines/iris/Evaluator/blessing/391.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 391 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'evaluation': [Artifact(artifact: uri: "pipelines/iris/Evaluator/evaluation/391"
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-29T16:30:37.469749:Evaluator:evaluation:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.4.0"
  }
}
, artifact_type: name: "ModelEvaluation"
)], 'blessing': [Artifact(artifact: uri: "pipelines/iris/Evaluator/blessing/391"
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-29T16:30:37.469749:Evaluator:blessing:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.4.0"
  }
}
, artifact_type: na

INFO:absl:Model on pipelines/iris/Evaluator/blessing/391 was not blessed by model validation
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 392 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'pushed_model': [Artifact(artifact: uri: "pipelines/iris/Pusher/pushed_model/392"
custom_properties {
  key: "name"
  value {
    string_value: "iris:2021-11-29T16:30:37.469749:Pusher:pushed_model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.4.0"
  }
}
, artifact_type: name: "PushedModel"
)]}) for execution 392
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Pusher is finished.


# Visualize pipeline

In [9]:
from tfx.orchestration.portable.mlmd import execution_lib
from tfx.orchestration.metadata import Metadata
from tfx.orchestration.experimental.interactive import visualizations
from ml_metadata.proto import metadata_store_pb2

def get_latest_artifact(component_id):
    metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(METADATA_PATH)

    with Metadata(metadata_connection_config) as metadata_handler:
        context = metadata_handler.store.get_context_by_type_and_name('node', f'{PIPELINE_NAME}.{component_id}')
        executions = metadata_handler.store.get_executions_by_context(context.id)
        latest_execution = max(executions, key=lambda e: e.last_update_time_since_epoch)

        return execution_lib.get_artifacts_dict(metadata_handler, latest_execution.id, [metadata_store_pb2.Event.OUTPUT])


def visualize(artifact):
    visualization = visualizations.get_registry().get_visualization(artifact.type_name)
    visualization.display(artifact)


from tfx.orchestration.experimental.interactive import standard_visualizations

standard_visualizations.register_standard_visualizations()

## StatisticsGen

In [10]:
statistic_artifact = get_latest_artifact("StatisticsGen")['statistics'][0]

visualize(statistic_artifact)

INFO:absl:MetadataStore with DB connection initialized


## SchemaGen

In [11]:
schema_artifact = get_latest_artifact("SchemaGen")['schema'][0]

visualize(schema_artifact)

INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'class',STRING,required,,'class'
'petallength',FLOAT,required,,-
'petalwidth',FLOAT,required,,-
'sepallength',FLOAT,required,,-
'sepalwidth',FLOAT,required,,-


,Values
Domain,
'class',"'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'"


## ExampleValidator

In [12]:
example_validator_articaft = get_latest_artifact("ExampleValidator")['anomalies'][0]

visualize(example_validator_articaft)

INFO:absl:MetadataStore with DB connection initialized


## Tuner

In [13]:
tuner_articaft = get_latest_artifact("Tuner")["best_hyperparameters"][0]

tuner_tensorboard = os.path.join(
    tuner_articaft.uri[0:(len('Tuner') + tuner_articaft.uri.index('Tuner'))],
    "tensorboards",
    os.path.split(tuner_articaft.uri.strip("/"))[-1]
)


%tensorboard --logdir {tuner_tensorboard}

INFO:absl:MetadataStore with DB connection initialized


## Trainer

In [14]:
model_run_artifact_dir = get_latest_artifact("Trainer")['model_run'][0].uri

%tensorboard --logdir {model_run_artifact_dir}

INFO:absl:MetadataStore with DB connection initialized


## Evaluator

In [15]:
schema_artifact = get_latest_artifact("Evaluator")['evaluation'][0]

visualize(schema_artifact)

INFO:absl:MetadataStore with DB connection initialized


In [16]:
# Get the TFMA output result path and load the result.
tfma_result = tfma.load_eval_result(schema_artifact.uri)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

# Serving

In [17]:
from IPython.display import Markdown as md
from IPython.display import display

serving_cmd = !pwd
serving_cmd = f"tensorflow_model_server --rest_api_port=8501 --model_name={PIPELINE_NAME} --model_base_path={os.path.join(serving_cmd[0], 'serving_model', PIPELINE_NAME)}"

display(md("# Serve"))
display(md("Serve the resulting model by running this in your shell:"))
display(md(f"``$ {serving_cmd}``"))

# Serve

Serve the resulting model by running this in your shell:

``$ tensorflow_model_server --rest_api_port=8501 --model_name=iris --model_base_path=/home/ekrem/Workplace/Python/tfx_seminar/serving_model/iris``

# Prediction

In [18]:
import json
import base64
import requests

URL = 'http://localhost:8501/v1/models/' + PIPELINE_NAME + ':predict'
HEADERS = {"content-type": "application/json"}

def prediction_request(sepallength, sepalwidth, petallength, petalwidth):
    features = {
        'petallength': tf.train.Feature(float_list=tf.train.FloatList(value=[petallength])),
        'petalwidth': tf.train.Feature(float_list=tf.train.FloatList(value=[petalwidth])),
        'sepallength': tf.train.Feature(float_list=tf.train.FloatList(value=[sepallength])),
        'sepalwidth': tf.train.Feature(float_list=tf.train.FloatList(value=[sepalwidth])),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=features))
    examples = example_proto.SerializeToString()

    request_data = json.dumps({
        "signature_name": "serving_default",
        "instances": [{
            "examples": {"b64": base64.b64encode(examples).decode('utf-8')}
        }]
    })

    response = requests.post(data=request_data, url=URL, headers=HEADERS)

    return json.loads(response.text)


In [20]:
prediction_request(sepallength=4.4,
                   sepalwidth=3.2,
                   petallength=1.3,
                   petalwidth=0.2)

{'predictions': [[1.0, 2.5669555e-11, 0.0]]}